In [72]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,sigma_i,sigma_ii,cor_i,cor_ii,capacities
import numpy as np
from util import *
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est

In [2]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [3]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [4]:
def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
    return fsolve(func, x0 = np.array([0.5, 0.5]))

In [15]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = chi[0]**2/sigma_i**4
cor_ii = chi[1]**2/sigma_ii**4

In [14]:
    P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer,prop_all_g_prefer,sigma_i,sigma_ii,cor_i,cor_ii)


In [25]:
cutoff_values = [P_A,P_B]

In [19]:
welfare_metrics([P_A,P_B],grade_estimated,chi,sigma,stud_pref)

Proportion of students in group 0 with no offer 0.5475
Proportion of students in group 0 with only a second preference offer 0.1125
Proportion of students in group 0 with first choice offer 0.34
Proportion of students in group 1 with no offer 0.615
Proportion of students in group 1 with only a second preference offer 0.14333333333333334
Proportion of students in group 1 with first choice offer 0.24166666666666667


In [70]:
student_by_col(cutoff_values,grade_estimated,chi,sigma,stud_pref)

Number of student admitted to college 1 in group 0: 99
Number of student admitted to college 2 in group 0: 83
Number of student admitted to college 1 in group 1: 149
Number of student admitted to college 2 in group 1: 83


(248, 167)

In [73]:
print(capacities)

[300, 200]
